In [1]:

%reload_ext XXX.mumble

In [2]:

    from poser import *
    from notebook.utils import url_path_join as ujoin
    from importlib import reload
    import requests
    import requests_cache  

In [3]:
    
    get_ipython().ast_node_interactivity='last_expr_or_assign'
    requests_cache.install_cache("election")
    get = requests.get;

In [52]:

    ---
    urls:
        prefix: http://results.enr.clarityelections.com/GA/
        reports: http://results.enr.clarityelections.com/GA/63991/184321/en/reports.html
        ga: http://results.enr.clarityelections.com/GA/
        elections: http://results.enr.clarityelections.com/GA/elections.json
        county: http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html
        franklin: http://results.enr.clarityelections.com/GA/Franklin/64051/
        franklin_redir: http://results.enr.clarityelections.com/GA/Franklin/64051/183122/en/summary.html
        json: http://results.enr.clarityelections.com/GA/63991/184321/json/status.json
        select: http://results.enr.clarityelections.com/GA/63991/184321/en/select-county.html
        wiki: https://en.wikipedia.org/wiki/List_of_counties_in_Georgia_(U.S._state)

In [36]:
    
    num_counties = (a[get][x.text][the.pandas.read_html()].drop(2).first().len()[x-1])(urls['wiki'])
    print(f"{num_counties} Counties")

159 Counties


In [37]:
    
    # A List of All of the Counties

    soup = a[get][x.text].bs4.BeautifulSoup(features='lxml')(urls['county'])
    county = (a.map(x.attrs['value']).list())(soup.select('table tr li a'));
    assert len(county) is num_counties, """The number of rows is not the number of counties."""

# The County Event IDs
The General election is made of a number of county-level events, which in turn have precinct-level
reporting.

In [39]:
    
    event_ids = a.map(a.str.split('/').filter(bool).take(2)).dict()(county);
    a.dict.items().take(3).dict()(event_ids)

{'Appling': '63993', 'Atkinson': '63994', 'Bacon': '63995'}

In [40]:

    def construct_context_version(name='Appling', sleep=1):
        latest_version = a[[a.identity(), event_ids.__getitem__]][
            [urls["prefix"]], a.identity(), ['current_ver.txt']
        ].concat()['/'.join][get][x.text]

        constructed = ujoin(urls["prefix"], name, event_ids[name], latest_version(name))
        constructed = ujoin(constructed , 'json/{}.json').format
        return a.map(constructed).map((
            a.str.rsplit('/', 1).last().str.replace('.json', ''), 
            a[get].methodcaller('json'))).dict()(('sum', 'details', 'status'))

In [41]:

    ---
    labels:
        CH: Candidate
        C: Office
        V: Votes
        TV: Total Voters
        BC: Ballot Cast

In [42]:

    def tidy_precinct(s):
        return a.zip().dict().valmap(
            a.zip(s.Candidate).dict()
        ).pandas.DataFrame()(s.P, s.V)

## Getting the Precinct-level data

In [49]:

    print("Loading precinct data...")
    dfs = {}
    for i, county in the.tqdm.tqdm_notebook()(list(enumerate(event_ids))):
        objects = construct_context_version(county)
        
        df = (a.pandas.DataFrame())(objects['sum']['Contests'])
        
        # assert df.V.apply(sum).lt(df.TV).all()
        df = df[list(labels)].rename(columns=labels)
        df = (a.pandas.DataFrame()[df.join])(objects['details']['Contests'])

        dfs[county] = a.pandas.concat()({index: tidy_precinct(row)
            for index, row in df.set_index('Office').iterrows() 
            if (a**list)(row.P) or not a.pandas.np.isnan()(row.P)})

Loading precinct data...


HBox(children=(IntProgress(value=0, max=159), HTML(value='')))

In [51]:

    tidy = a.pandas.concat()(dfs)

    tidy = tidy.swaplevel(0,1).stack().pipe(
        lambda df: a.pandas.Series()(df.values, df.index.rename(('contest', 'county', 'candidate', 'precinct')))
    ).rename('votes')
    tidy.sample(3)

contest                                          county      candidate              precinct        
U.S. Representative, District 12                 Jeff_Davis  PATRICIA C. MCCRACKEN  Altamaha 2            48.0
Public Service Commission, District 2 - Eastern  Fulton      TIM ECHOLS (I)         AP09A               1835.0
Probate Judge                                    Walton      B. E. WRIGHT (I)(R)    Loganville North    2556.0
Name: votes, dtype: float64

In [48]:

    the.pandas.DataFrame()(tidy).sample(3)

,,,,votes
contest,county,candidate,precinct,
President of the United States,Richmond,DONALD J. TRUMP,306,861.0
SHERIFF,Randolph,E. FAIRBANKS (I) D,Springvale,147.0
"JUDGE, PROBATE COURT",Gilmer,S. C. CHASTAIN (R),BIG CREEK,761.0
